In [1]:
import pandas as pd
import mysql.connector as sql
import math

In [2]:
def get_all_bonds_in_list(start_date,end_date):
    print('start to get data from {} to {}'.format(start_date,end_date))
    #####Create engine:
    db_connection = sql.connect(host='0.0.0.0', database='bond_db', user='root', password='password')
    ##security query
    security_query = "select * from security_info"
    ##read data
    security_data = pd.read_sql(security_query,con=db_connection)
    security_data = security_data[['SecurityID','Currency','IssueDate','MaturityDate']]
    ##price query
    price_query = "select * from bond_spread where  KeyDate between '{}' and '{}'".format(start_date,end_date)
    #####read bond_data from db:
    final_data = pd.read_sql(price_query,con=db_connection)
    final_data = final_data.merge(security_data, on=['SecurityID'], how='left')
    final_data.dropna(subset=["ZSpread"],inplace=True)
    final_data_1 = final_data[final_data.Currency == 'USD']
    final_data = final_data_1.groupby('SecurityID')
    bonds_list = [final_data.get_group(x) for x in final_data.groups]
    db_connection.close()
    return bonds_list

In [3]:
def merge_left_right(data1,data2):
    temp_data = data1.append(data2)
    return temp_data

def merg_sort(list_data):
    list_length = len(list_data)
    if list_length == 1:
        output = list_data[0][2]
        return output
    else:
        mid_point = math.floor(list_length/2)
        #print(len(list_data[:mid_point]))
        left = list_data[mid_point:]
        right = list_data[:mid_point]
        return merge_left_right(merg_sort(left), merg_sort(right))

In [4]:
silding_windows = [['2018-12-31','2019-02-28'],['2019-03-01','2019-04-30'],
                   ['2019-05-01','2019-06-30'],['2019-07-01','2019-08-31'],
                   ['2019-09-01','2019-10-31'],['2019-11-01','2019-12-31']]

## Cluster data

In [5]:
cluster_data = pd.read_csv('Cluster_group_125.csv')

In [6]:
cluster_data.drop(columns=['Unnamed: 0'],inplace=True)

In [7]:
cluster_data.head()

,SecurityID,2019-02-28,2019-04-30,2019-06-30,2019-08-31,2019-10-31,2019-12-31
0,3051.0,Cluster 0,Cluster 85,Cluster 70,Cluster 82,Cluster 99,Cluster 91
1,3704.0,Cluster 0,Cluster 119,Cluster 0,Cluster 12,Cluster 65,Cluster 87
2,4121.0,Cluster 0,Cluster 52,Cluster 53,Cluster 72,Cluster 6,Cluster 91
3,4130.0,Cluster 0,Cluster 76,Cluster 124,Cluster 89,Cluster 10,Cluster 109
4,4259.0,Cluster 0,Cluster 95,Cluster 28,Cluster 86,Cluster 105,Cluster 76


In [29]:
def get_the_weekly_spread_windows(bonds_list):
    bond_spread_list = []
    for bond in bonds_list:
        try:
            bond['KeyDate'] = pd.to_datetime(bond['KeyDate'])
            bond = bond.set_index('KeyDate')
            logic = {'GSpread'  : 'last',
                    'ModifiedDuration_Plain':'last',
                    'YieldWorst':'last',
                    'AmtOutstanding':'last',
                 'SecurityID':'first'}
            offset = pd.offsets.timedelta(days=-6)
            d = bond.resample('W', loffset=offset).apply(logic)
            d['G_change_Percent']=d.GSpread.pct_change()
            d['G_change'] = d.GSpread.diff()
            d['ModifiedDuration_Plain_change']=d.ModifiedDuration_Plain.pct_change()
            d['YieldWorst_change']=d.YieldWorst.pct_change()
            d.dropna(subset=['G_change_Percent'],inplace = True)
            d.reset_index(inplace=True)
            bond_spread_list.append((len(d.GSpread.values),bond.SecurityID.iloc[0],d))
        except:
            pass
    return bond_spread_list

In [9]:
def Average_daily_Gspread_change_cluster(window,cluster_data):
    ####Grab Data###
    bond_list1 = get_all_bonds_in_list(window[0],window[1])
    ####G Change ###
    daily_spread_1 = get_the_weekly_spread_windows(bond_list1)
    new_data = merg_sort(daily_spread_1)
    new_data.dropna(inplace=True)
    data1 = new_data.merge(cluster_data[['SecurityID',window[1]]],on=['SecurityID'],how ='left')
    temp = data1.groupby(by=['KeyDate',window[1]],as_index=False).mean()
    data1['Group'] = data1[window[1]] 
    data1.drop(columns=[window[1]],inplace=True)
    temp['Group'] = temp[window[1]]
    end = temp[['Group','KeyDate','G_change_Percent','YieldWorst_change','ModifiedDuration_Plain_change']]
    end = end.rename(columns={'G_change_Percent':'Cluster_G_change','YieldWorst_change':'Cluster_average_YieldWorst_change','ModifiedDuration_Plain_change':'Cluster_average_ModifiedDuration_Plain_change'})
    return end,data1

In [10]:
db_connection = sql.connect(host='0.0.0.0', database='bond_db', user='root', password='password')
security_query = "select * from security_info"
security_data = pd.read_sql(security_query,con=db_connection)

## END

## Read Your Rating data

In [11]:
def term(x,y):
    try:
        bond_term = int(x.year) - int(y.year)
    except:
        bond_term = 100
    return bond_term

In [12]:
def Type_term(x):
    if x >= 10:
        return 'Long Term'
    elif x < 10 and x >= 5:
        return  'Mid Term'
    else:
        return 'Short Term'

In [13]:
def assign_rating(x):
    x = float(x)
    if math.isnan(x):
        return 'UN'
    else:
        if x < 0:
            return 'NR'
        elif x<=13.5:
            return "Junk"
        elif x>13.5 and x<=16.5:
            return 'Lower medium grade'
        elif x>16.5 and x<=19.5:
            return 'Upper medium grade'
        elif x>19.5 and x<=22.5:
            return 'High grade'
        else:
            return 'Prime'

In [14]:
db_connection = sql.connect(host='0.0.0.0', database='bond_db', user='root', password='password')
rating_query = "select * from rating"
rating_data = pd.read_sql(rating_query,con=db_connection)

In [15]:
rating_data = rating_data.groupby(by=['SecurityID'],as_index=False).mean()

In [16]:
security_data['Term'] = security_data.apply(lambda x:term(x.MaturityDate,x.IssueDate),axis=1)

In [17]:
security_data['Term'] = security_data['Term'].fillna(100)
security_data['Type of Term'] = security_data['Term'].apply(Type_term)

In [18]:
security_data = security_data.merge(rating_data,on =['SecurityID'],how='left')

In [19]:
security_data.head()

,SecurityID,OfficialName,SecurityType,SecuritySubType,SecurityBBGType,SecurityBBGYellowKey,Series,SecuritySector,SecurityIndustry,SecuritySubIndustry,...,PaymentRank,IsBailIn,QuoteSize,AmountIssued,MinIncrement,MinPiece,BBGDescription,Term,Type of Term,RatingSP
0,62,BACR 4.875 08/13/19,Bond,Bond,EURO MTN,Corp,None,Financials,Banks,Commer Banks Non-US,...,Sr Unsecured,N,100,2.000000e+09,1000.0,50000.0,None,10,Long Term,NaN
1,64,POST 5 08/15/26,Bond,Bond,PRIV PLACEMENT,Corp,None,Consumers,Food & Beverage,Food-Misc/Diversified,...,Sr Unsecured,Not Applicable,100,1.750000e+09,1000.0,2000.0,None,10,Long Term,10.000000
2,69,ARNDTN 1.5 05/03/22,Bond,Bond,EURO-ZONE,Corp,None,Real Estate,Real Estate,Real Estate Oper/Develop,...,Sr Unsecured,Not Applicable,100,6.000000e+08,100000.0,100000.0,COUPON STEP UP PROVISION IF ISSUER IS NOT BBB ...,6,Mid Term,NaN
3,72,RBS 6.1 06/10/2023 Bullet USD SUBORDINATE,Bond,Bond,GLOBAL,Corp,None,Financials,Diversified Banks,Diversified Banking Inst,...,Subordinated,Y,100,1.000000e+09,1000.0,2000.0,Basel III grandfathered instrument,10,Long Term,13.000000
4,78,AER 5 10/01/21,Bond,Bond,GLOBAL,Corp,None,Financials,Commercial Finance,Finance-Leasing Compan,...,Sr Unsecured,N,100,8.000000e+08,1000.0,150000.0,ISS'D IN EXCH OF 144A/REGS SEC; SEE CUSIP# 007...,6,Mid Term,14.652361


In [20]:
security_data['Rating'] = security_data['RatingSP'].apply(assign_rating)

In [21]:
security_data.head()

,SecurityID,OfficialName,SecurityType,SecuritySubType,SecurityBBGType,SecurityBBGYellowKey,Series,SecuritySector,SecurityIndustry,SecuritySubIndustry,...,IsBailIn,QuoteSize,AmountIssued,MinIncrement,MinPiece,BBGDescription,Term,Type of Term,RatingSP,Rating
0,62,BACR 4.875 08/13/19,Bond,Bond,EURO MTN,Corp,None,Financials,Banks,Commer Banks Non-US,...,N,100,2.000000e+09,1000.0,50000.0,None,10,Long Term,NaN,UN
1,64,POST 5 08/15/26,Bond,Bond,PRIV PLACEMENT,Corp,None,Consumers,Food & Beverage,Food-Misc/Diversified,...,Not Applicable,100,1.750000e+09,1000.0,2000.0,None,10,Long Term,10.000000,Junk
2,69,ARNDTN 1.5 05/03/22,Bond,Bond,EURO-ZONE,Corp,None,Real Estate,Real Estate,Real Estate Oper/Develop,...,Not Applicable,100,6.000000e+08,100000.0,100000.0,COUPON STEP UP PROVISION IF ISSUER IS NOT BBB ...,6,Mid Term,NaN,UN
3,72,RBS 6.1 06/10/2023 Bullet USD SUBORDINATE,Bond,Bond,GLOBAL,Corp,None,Financials,Diversified Banks,Diversified Banking Inst,...,Y,100,1.000000e+09,1000.0,2000.0,Basel III grandfathered instrument,10,Long Term,13.000000,Junk
4,78,AER 5 10/01/21,Bond,Bond,GLOBAL,Corp,None,Financials,Commercial Finance,Finance-Leasing Compan,...,N,100,8.000000e+08,1000.0,150000.0,ISS'D IN EXCH OF 144A/REGS SEC; SEE CUSIP# 007...,6,Mid Term,14.652361,Lower medium grade


In [22]:
def combine_data(List1,num_of_clusters):
    final = []
    length = len(List1)
    for j in range(0,length):   
        for i in range(0,num_of_clusters):
            if i == 0:
                f = List1[j].loc['Cluster {}'.format(i)].T.head(1).reset_index().iloc[:,1:]
                f['Group'] ='Cluster {}'.format(i)
            else:
                temp = List1[j].loc['Cluster {}'.format(i)].T.head(1).reset_index().iloc[:,1:]
                temp['Group'] ='Cluster {}'.format(i)
                f = f.merge(temp,how='outer').fillna(0)
        final.append(f)
    for i in range(0,len(final)):
        if i == 0:
            f_data = final[i]
        else:
            f_data = f_data.merge(final[i],on='Group',how = 'left')
    return f_data

In [23]:
def get_the_cluster_data(security_info,cluster_data,windows):
    print('start to get data for {}'.format(windows[1]))
    security_info = security_data[['SecurityID','SecuritySector','AmountIssued','Coupon','Seniority','Term','Type of Term','RatingSP','Rating']]
    a = security_info.merge(cluster_data[['SecurityID',windows[1]]],on=['SecurityID'],how='left')
    a.dropna(subset=[windows[1]],inplace=True)
    a = a.rename(columns={windows[1]:'Group'})
    c = a.groupby(by=['Group','Seniority']).count()
    s= a.groupby(by=['Group','SecuritySector']).count()	
    r = a.groupby(by=['Group','Rating']).count()
    t = a.groupby(by=['Group','Type of Term']).count()
    d = a.groupby(by=['Group']).count()
    e = c/d
    e1 = s/d
    e2 = r/d
    e3 = t/d
    list1 = [e,e1,e2,e3]
    f = combine_data(list1,125)
    b = a.groupby(by=['Group'],as_index=False).mean()
    b.drop(columns=['SecurityID'],inplace=True)
    b = b.rename(columns={'AmountIssued':'Cluster_average_AmountIssued','Coupon':'Cluster_average_Coupon','Term':'Cluster_average_Term','RatingSP':'Cluster_average_Rating'})
    window_cluster_data = b.merge(f,on='Group',how = 'left')
    return window_cluster_data


In [24]:
from sqlalchemy import create_engine

In [30]:
for i in range(0,len(silding_windows)):
    if i == 0:
        final_change,final = Average_daily_Gspread_change_cluster(silding_windows[i],cluster_data)
        temp_cluster = get_the_cluster_data(security_data,cluster_data,silding_windows[i])
        final = final.merge(temp_cluster, on='Group',how='left')
    else:
        temp,temp_data = Average_daily_Gspread_change_cluster(silding_windows[i],cluster_data)
        temp_cluster = get_the_cluster_data(security_data,cluster_data,silding_windows[i])
        temp_data = temp_data.merge(temp_cluster, on='Group',how='left')
        final_change = final_change.append(temp,ignore_index=True)
        final = final.append(temp_data,ignore_index=True)

start to get data from 2018-12-31 to 2019-02-28
start to get data for 2019-02-28
start to get data from 2019-03-01 to 2019-04-30
start to get data for 2019-04-30
start to get data from 2019-05-01 to 2019-06-30
start to get data for 2019-06-30
start to get data from 2019-07-01 to 2019-08-31
start to get data for 2019-08-31
start to get data from 2019-09-01 to 2019-10-31
start to get data for 2019-10-31
start to get data from 2019-11-01 to 2019-12-31
start to get data for 2019-12-31


In [31]:
final_change

,Group,KeyDate,Cluster_G_change,Cluster_average_YieldWorst_change,Cluster_average_ModifiedDuration_Plain_change
0,Cluster 0,2019-01-07,-0.084004,-0.008013,-0.003674
1,Cluster 1,2019-01-07,-0.077047,-0.008904,-0.002232
2,Cluster 10,2019-01-07,-0.086713,-0.018195,-0.000821
3,Cluster 100,2019-01-07,-0.038036,0.004060,-0.004694
4,Cluster 101,2019-01-07,-0.026352,0.012550,-0.005123
...,...,...,...,...,...
6245,Cluster 95,2019-12-30,-0.021033,0.003518,-0.003206
6246,Cluster 96,2019-12-30,0.006330,0.005908,-0.005893
6247,Cluster 97,2019-12-30,-0.001332,0.004865,-0.002280
6248,Cluster 98,2019-12-30,-0.042790,-0.002651,-0.006107


In [34]:
final.columns

Index(['KeyDate', 'GSpread', 'ModifiedDuration_Plain', 'YieldWorst',
       'AmtOutstanding', 'SecurityID', 'G_change_Percent', 'G_change',
       'ModifiedDuration_Plain_change', 'YieldWorst_change', 'Group',
       'Cluster_average_AmountIssued', 'Cluster_average_Coupon',
       'Cluster_average_Term', 'Cluster_average_Rating',
       'Jr Subordinated Unsecured', 'Secured', 'Sr Unsecured',
       'Subordinated Unsecured', '1st Lien Secured', '2nd Lien Secured',
       '3rd Lien Secured', '1st lien', 'Auto', 'Basic Materials',
       'Communications', 'Consumers', 'Energy', 'Financials', 'Health Care',
       'Industrials', 'Real Estate', 'Technology', 'High grade', 'Junk',
       'Lower medium grade', 'UN', 'Upper medium grade', 'Prime', 'Long Term',
       'Mid Term', 'Short Term', 'Diversified', 'Asset Backed'],
      dtype='object')

In [35]:
len(final)

758189

In [36]:
final = final[['SecurityID','KeyDate','YieldWorst','ModifiedDuration_Plain','AmtOutstanding','G_change','YieldWorst_change',
                'ModifiedDuration_Plain_change','Group','Cluster_average_AmountIssued',
                'Cluster_average_Coupon', 'Cluster_average_Term',
                'Cluster_average_Rating', 'Jr Subordinated Unsecured', 'Secured',
                'Sr Unsecured', 'Subordinated Unsecured', '1st Lien Secured',
                '2nd Lien Secured', '3rd Lien Secured', '1st lien', 'Asset Backed',
                'Auto', 'Basic Materials','Communications', 'Consumers', 'Energy', 'Financials', 'Health Care',
                'Industrials', 'Real Estate', 'Technology', 'High grade', 'Junk',
                'Lower medium grade', 'UN', 'Upper medium grade', 'Prime', 'Long Term',
                'Mid Term', 'Short Term']]

In [38]:
final = final.rename(columns={ 'Jr Subordinated Unsecured':'Cluster Perecentage of Jr Subordinated Unsecured',
                               'Secured':'Cluster Perecentage of Secured',
                               'Sr Unsecured':'Cluster Perecentage of Sr Unsecured', 
                               'Subordinated Unsecured':'Cluster Perecentage of Subordinated Unsecured', 
                               '1st Lien Secured':'Cluster Perecentage of 1st Lien Secured',
                               '2nd Lien Secured':'Cluster Perecentage of 2nd Lien Secured', 
                               '3rd Lien Secured':'Cluster Perecentage of 3rd Lien Secured', 
                               '1st lien':'Cluster Perecentage of 1st lien', 
                               'Asset Backed':'Cluster Perecentage of Asset Backed',
                               'Auto':'Cluster Perecentage of Auto', 
                               'Basic Materials':'Cluster Perecentage of Basic Materials',
                               'Communications':'Cluster Perecentage of Communications', 
                               'Consumers':'Cluster Perecentage of Consumers', 
                               'Energy':'Cluster Perecentage of Energy', 
                               'Financials':'Cluster Perecentage of Financials', 
                               'Health Care':'Cluster Perecentage of Health Care',
                               'Industrials':'Cluster Perecentage of Industrials', 
                               'Real Estate':'Cluster Perecentage of Real Estate', 
                               'Technology':'Cluster Perecentage of Technology', 
                               'High grade':'Cluster Perecentage of High grade', 
                               'Junk':'Cluster Perecentage of Junk',
                               'Lower medium grade':'Cluster Perecentage of Lower medium grade', 
                               'UN':'Cluster Perecentage of UN', 
                               'Upper medium grade':'Cluster Perecentage of Upper medium grade',
                               'Prime':'Cluster Perecentage of Prime', 
                               'Long Term':'Cluster Perecentage of Long Term',
                               'Mid Term':'Cluster Perecentage of Mid Term', 
                               'Short Term':'Cluster Perecentage of Short Term'})



In [39]:
final = final.rename(columns={"G_change": "Target_G_change"})

In [40]:
final1 =final

In [41]:
final = final.merge(final_change,on=['Group','KeyDate'],how = 'left')

In [42]:
final.dropna(subset=['Group'],inplace=True)

In [43]:
final.columns

Index(['SecurityID', 'KeyDate', 'YieldWorst', 'ModifiedDuration_Plain',
       'AmtOutstanding', 'Target_G_change', 'YieldWorst_change',
       'ModifiedDuration_Plain_change', 'Group',
       'Cluster_average_AmountIssued', 'Cluster_average_Coupon',
       'Cluster_average_Term', 'Cluster_average_Rating',
       'Cluster Perecentage of Jr Subordinated Unsecured',
       'Cluster Perecentage of Secured', 'Cluster Perecentage of Sr Unsecured',
       'Cluster Perecentage of Subordinated Unsecured',
       'Cluster Perecentage of 1st Lien Secured',
       'Cluster Perecentage of 2nd Lien Secured',
       'Cluster Perecentage of 3rd Lien Secured',
       'Cluster Perecentage of 1st lien',
       'Cluster Perecentage of Asset Backed', 'Cluster Perecentage of Auto',
       'Cluster Perecentage of Basic Materials',
       'Cluster Perecentage of Communications',
       'Cluster Perecentage of Consumers', 'Cluster Perecentage of Energy',
       'Cluster Perecentage of Financials',
       'Clu

In [44]:
final = final.fillna(0)

In [45]:
len(final)

724376

In [46]:
final[final['SecurityID'] ==64]

,SecurityID,KeyDate,YieldWorst,ModifiedDuration_Plain,AmtOutstanding,Target_G_change,YieldWorst_change,ModifiedDuration_Plain_change,Group,Cluster_average_AmountIssued,...,Cluster Perecentage of Lower medium grade,Cluster Perecentage of UN,Cluster Perecentage of Upper medium grade,Cluster Perecentage of Prime,Cluster Perecentage of Long Term,Cluster Perecentage of Mid Term,Cluster Perecentage of Short Term,Cluster_G_change,Cluster_average_YieldWorst_change,Cluster_average_ModifiedDuration_Plain_change
108541,64.0,2019-01-07,5.91076,6.07625,1.697270e+09,-52.350,-0.066614,0.002569,Cluster 107,7.072015e+08,...,0.275591,0.023622,0.149606,0.000000,0.614173,0.362205,0.023622,-0.105282,-0.037501,0.001981
108542,64.0,2019-01-14,5.72403,6.07073,1.697270e+09,-28.400,-0.031592,-0.000908,Cluster 107,7.072015e+08,...,0.275591,0.023622,0.149606,0.000000,0.614173,0.362205,0.023622,-0.063508,-0.014419,-0.003099
108543,64.0,2019-01-21,5.86272,6.04487,1.697270e+09,20.040,0.024229,-0.004260,Cluster 107,7.072015e+08,...,0.275591,0.023622,0.149606,0.000000,0.614173,0.362205,0.023622,0.057335,0.014442,-0.004128
108544,64.0,2019-01-28,5.80981,6.03392,1.697270e+09,-0.440,-0.009025,-0.001811,Cluster 107,7.072015e+08,...,0.275591,0.023622,0.149606,0.000000,0.614173,0.362205,0.023622,-0.030705,-0.024576,0.000210
108545,64.0,2019-02-04,5.83174,6.01464,1.697270e+09,7.310,0.003775,-0.003195,Cluster 107,7.072015e+08,...,0.275591,0.023622,0.149606,0.000000,0.614173,0.362205,0.023622,-0.004846,-0.012482,-0.002300
108546,64.0,2019-02-11,5.67192,6.12945,1.697270e+09,-19.370,-0.027405,0.019088,Cluster 107,7.072015e+08,...,0.275591,0.023622,0.149606,0.000000,0.614173,0.362205,0.023622,-0.038582,-0.010549,-0.002442
108547,64.0,2019-02-18,5.66178,6.11500,1.697270e+09,0.640,-0.001788,-0.002357,Cluster 107,7.072015e+08,...,0.275591,0.023622,0.149606,0.000000,0.614173,0.362205,0.023622,-0.014700,-0.010173,-0.002856
108548,64.0,2019-02-25,5.65774,6.09504,1.697270e+09,-7.500,-0.000714,-0.003264,Cluster 107,7.072015e+08,...,0.275591,0.023622,0.149606,0.000000,0.614173,0.362205,0.023622,-0.032302,-0.005922,-0.003034
239085,64.0,2019-03-04,5.70859,6.07115,1.697270e+09,25.620,0.019866,-0.004215,Cluster 80,8.381850e+08,...,0.455090,0.017964,0.323353,0.000000,0.826347,0.167665,0.005988,0.034905,-0.013627,-0.000910
239086,64.0,2019-03-11,5.54003,6.06551,1.697270e+09,-13.520,-0.029527,-0.000929,Cluster 80,8.381850e+08,...,0.455090,0.017964,0.323353,0.000000,0.826347,0.167665,0.005988,-0.015036,-0.012801,0.000532


## Produce Unscaled Security Dummpy

In [32]:
security_data.head()

,SecurityID,OfficialName,SecurityType,SecuritySubType,SecurityBBGType,SecurityBBGYellowKey,Series,SecuritySector,SecurityIndustry,SecuritySubIndustry,...,Seniority,PaymentRank,IsBailIn,QuoteSize,AmountIssued,MinIncrement,MinPiece,BBGDescription,Term,RatingSP
0,62,BACR 4.875 08/13/19,Bond,Bond,EURO MTN,Corp,None,Financials,Banks,Commer Banks Non-US,...,Sr Unsecured,Sr Unsecured,N,100,2.000000e+09,1000.0,50000.0,None,10,NaN
1,64,POST 5 08/15/26,Bond,Bond,PRIV PLACEMENT,Corp,None,Consumers,Food & Beverage,Food-Misc/Diversified,...,Sr Unsecured,Sr Unsecured,Not Applicable,100,1.750000e+09,1000.0,2000.0,None,10,10.000000
2,69,ARNDTN 1.5 05/03/22,Bond,Bond,EURO-ZONE,Corp,None,Real Estate,Real Estate,Real Estate Oper/Develop,...,Sr Unsecured,Sr Unsecured,Not Applicable,100,6.000000e+08,100000.0,100000.0,COUPON STEP UP PROVISION IF ISSUER IS NOT BBB ...,6,NaN
3,72,RBS 6.1 06/10/2023 Bullet USD SUBORDINATE,Bond,Bond,GLOBAL,Corp,None,Financials,Diversified Banks,Diversified Banking Inst,...,Subordinated Unsecured,Subordinated,Y,100,1.000000e+09,1000.0,2000.0,Basel III grandfathered instrument,10,13.000000
4,78,AER 5 10/01/21,Bond,Bond,GLOBAL,Corp,None,Financials,Commercial Finance,Finance-Leasing Compan,...,Sr Unsecured,Sr Unsecured,N,100,8.000000e+08,1000.0,150000.0,ISS'D IN EXCH OF 144A/REGS SEC; SEE CUSIP# 007...,6,14.652361


In [33]:
len(security_data)

28976

In [34]:
security_data.columns

Index(['SecurityID', 'OfficialName', 'SecurityType', 'SecuritySubType',
       'SecurityBBGType', 'SecurityBBGYellowKey', 'Series', 'SecuritySector',
       'SecurityIndustry', 'SecuritySubIndustry', 'BBGSecuritySector',
       'IssuerParent', 'Obligor', 'Ticker', 'EquityTicker', 'Issuer',
       'ObligorID', 'CountryDomicile', 'CountryRisk', 'CountryIssue',
       'Currency', 'Coupon', 'CouponType', 'CouponFrequency', 'CouponDayCount',
       'CreationDate', 'LastModifiedTime', 'IssueDate', 'PricingDate',
       'MaturityDate', 'IsCallable', 'IsConvertible', 'IsContingent',
       'IsFactorable', 'IsHybrid', 'IsMaple', 'IsPrivatePlacement',
       'IsPreliminary', 'IsPerpetual', 'IsJunior', 'Seniority', 'PaymentRank',
       'IsBailIn', 'QuoteSize', 'AmountIssued', 'MinIncrement', 'MinPiece',
       'BBGDescription', 'Term', 'RatingSP'],
      dtype='object')

In [35]:
security_data.Seniority.unique()

array(['Sr Unsecured', 'Subordinated Unsecured',
       'Jr Subordinated Unsecured', 'Secured', '1st Lien Secured',
       '2nd Lien Secured', 'Asset Backed', '3rd Lien Secured', '1st lien',
       '1.5 Lien Secured'], dtype=object)

In [36]:
df = pd.concat([security_data['SecurityID'],pd.get_dummies(security_data.Seniority)],axis=1) 

In [37]:
df = pd.concat([df,pd.get_dummies(security_data.SecuritySector)],axis=1) 

In [38]:
df = pd.concat([df,pd.get_dummies(security_data.CountryRisk)],axis=1) 

In [39]:
df = pd.concat([df,pd.get_dummies(security_data.CouponType)],axis=1) 

In [40]:
df

,SecurityID,1.5 Lien Secured,1st Lien Secured,1st lien,2nd Lien Secured,3rd Lien Secured,Asset Backed,Jr Subordinated Unsecured,Secured,Sr Unsecured,...,FIXED,FLAT TRADING,FLOATING,FUNGED,Fixed,PAY-IN-KIND,STEP CPN,VARIABLE,WHEN ISSUED,ZERO COUPON
0,62,0,0,0,0,0,0,0,0,1,...,1,0,0,0,0,0,0,0,0,0
1,64,0,0,0,0,0,0,0,0,1,...,1,0,0,0,0,0,0,0,0,0
2,69,0,0,0,0,0,0,0,0,1,...,1,0,0,0,0,0,0,0,0,0
3,72,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,78,0,0,0,0,0,0,0,0,1,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28971,72948,0,0,0,0,0,0,0,0,1,...,1,0,0,0,0,0,0,0,0,0
28972,72949,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
28973,72950,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
28974,72951,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0


In [37]:
df_unsacled = pd.concat([df,security_data[['MinIncrement','MinPiece','Term','RatingSP']]],axis=1) 

In [39]:
df_unsacled

,SecurityID,1.5 Lien Secured,1st Lien Secured,1st lien,2nd Lien Secured,3rd Lien Secured,Asset Backed,Jr Subordinated Unsecured,Secured,Sr Unsecured,...,Fixed,PAY-IN-KIND,STEP CPN,VARIABLE,WHEN ISSUED,ZERO COUPON,MinIncrement,MinPiece,Term,RatingSP
0,62,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,1000.0,50000.0,10,NaN
1,64,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,1000.0,2000.0,10,10.000000
2,69,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,100000.0,100000.0,6,NaN
3,72,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1000.0,2000.0,10,13.000000
4,78,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,1000.0,150000.0,6,14.652361
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28971,72948,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,1000.0,200000.0,3,NaN
28972,72949,0,0,0,0,0,0,0,0,1,...,0,0,0,1,0,0,1000.0,200000.0,100,NaN
28973,72950,0,0,0,0,0,0,0,0,1,...,0,0,0,1,0,0,1000.0,200000.0,100,NaN
28974,72951,0,0,0,0,0,0,1,0,0,...,0,0,0,1,0,0,200000.0,200000.0,100,NaN


In [40]:
df_unsacled.to_csv('Security_info_post_dummy.csv')

In [59]:
##Market query
Market_query = "select * from market_data_weekly"
Market_data = pd.read_sql(Market_query,con=db_connection)

In [60]:
Market_data = Market_data.fillna(0)

In [61]:
Market_data

,KeyDate,skew_change,swap_change,vix_change,Close_change,slope_change,skew_change_Weekly,swap_change_Weekly,vix_change_Weekly,Close_change_Weekly,slope_change_Weekly
0,2019-01-02,-0.004584,-0.004966,-0.086546,0.001269,-0.238095,0.000000,0.0,0.00000,0.000000,-0.026617
1,2019-01-03,-0.042981,0.000000,0.096038,-0.024757,0.062500,0.000000,0.0,0.00000,0.000000,-0.024643
2,2019-01-04,-0.006951,-0.012668,-0.159921,0.034336,0.000000,0.000000,0.0,0.00000,0.000000,-0.034643
3,2019-01-07,0.019024,0.005443,0.000935,0.007010,0.000000,0.000000,0.0,0.00000,0.000000,-0.025119
4,2019-01-08,0.015322,0.020495,-0.043458,0.009695,-0.117647,-0.004034,0.0,-0.03859,0.005511,-0.058648
...,...,...,...,...,...,...,...,...,...,...,...
410,2020-08-18,0.000000,0.000000,0.000000,0.002303,0.000000,0.000000,0.0,0.00000,0.003358,0.000000
411,2020-08-19,0.000000,0.000000,0.000000,-0.004404,0.000000,0.000000,0.0,0.00000,-0.000322,0.000000
412,2020-08-20,0.000000,0.000000,0.000000,0.003159,0.000000,0.000000,0.0,0.00000,0.000719,0.000000
413,2020-08-21,0.000000,0.000000,0.000000,0.003441,0.000000,0.000000,0.0,0.00000,0.001442,0.000000


In [62]:
Market_data = Market_data[['KeyDate','skew_change_Weekly','swap_change_Weekly','vix_change_Weekly','Close_change_Weekly','slope_change_Weekly']]

In [63]:
final_change_data2 = final.merge(Market_data,on=['KeyDate'],how='left')

In [66]:
final_change_data2.fillna(0)

,SecurityID,KeyDate,YieldWorst,ModifiedDuration_Plain,AmtOutstanding,Target_G_change,YieldWorst_change,ModifiedDuration_Plain_change,Group,Cluster_average_AmountIssued,...,Cluster Perecentage of Mid Term,Cluster Perecentage of Short Term,Cluster_G_change,Cluster_average_YieldWorst_change,Cluster_average_ModifiedDuration_Plain_change,skew_change_Weekly,swap_change_Weekly,vix_change_Weekly,Close_change_Weekly,slope_change_Weekly
0,56535.0,2019-01-07,26.58950,1.83121,3.000000e+08,-49.590,-0.013582,-0.006677,Cluster 83,6.380922e+08,...,0.288889,0.211111,-0.049186,0.001261,-0.008545,0.000000,0.000000,0.000000,0.000000,-0.025119
1,56535.0,2019-01-14,21.54340,1.86436,3.000000e+08,-512.500,-0.189778,0.018103,Cluster 83,6.380922e+08,...,0.288889,0.211111,-0.049477,-0.001030,-0.010469,0.019833,0.004032,-0.022044,0.002582,0.019248
2,56535.0,2019-01-21,18.51400,1.88186,3.000000e+08,-299.500,-0.140618,0.009387,Cluster 83,6.380922e+08,...,0.288889,0.211111,-0.051169,-0.019856,-0.006177,0.000000,0.000000,0.000000,0.000000,0.000000
3,56535.0,2019-01-28,17.52680,1.87785,3.000000e+08,-90.430,-0.053322,-0.002131,Cluster 83,6.380922e+08,...,0.288889,0.211111,-0.059208,-0.026304,-0.006901,-0.011853,-0.002704,0.016029,-0.001987,-0.021487
4,56535.0,2019-02-04,17.76590,1.85843,3.000000e+08,27.600,0.013642,-0.010342,Cluster 83,6.380922e+08,...,0.288889,0.211111,-0.043795,-0.023787,-0.008677,0.008622,-0.003730,-0.035228,0.006073,0.060556
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
724371,64.0,2019-12-02,3.47623,5.60152,1.697270e+09,2.754,0.005275,-0.003294,Cluster 88,6.523597e+08,...,0.297297,0.153153,0.038067,0.013617,-0.013243,0.001454,0.000000,0.041562,0.000247,0.077514
724372,64.0,2019-12-09,3.39679,5.58468,1.697270e+09,-6.099,-0.022852,-0.003006,Cluster 88,6.523597e+08,...,0.297297,0.153153,-0.031402,-0.017380,-0.011350,0.016157,0.005062,0.016261,0.001432,-0.012690
724373,64.0,2019-12-16,3.06738,5.57198,1.697270e+09,-33.788,-0.096977,-0.002274,Cluster 88,6.523597e+08,...,0.297297,0.153153,-0.033877,-0.001036,-0.013194,0.011185,0.006814,-0.051634,0.003521,0.050238
724374,64.0,2019-12-23,2.96338,5.55818,1.697270e+09,-6.522,-0.033905,-0.002477,Cluster 88,6.523597e+08,...,0.297297,0.153153,-0.024797,-0.020291,-0.010527,0.000777,0.000000,0.007677,0.002035,0.039974


In [67]:
final_change_data2.to_csv('Security_and_market_movement_unscaled_cluster_weekly_125.csv')

In [68]:
len(final_change_data2)

724376